In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## close range vrs distance range relative to the trollinde radar.

In [2]:
track = pd.read_csv('../saved-files/threshold-all/final-tracks-threshold-all.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,311,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,3,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,463,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,3,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,610,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,3,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,299,43,144.743717,239.312025,4,15,35461,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,12303,0 days 00:05:00,122.013418,15.240493,41.116443,37.803245
25142,300,49,146.461260,240.148842,5,15,35519,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,12303,0 days 00:10:00,120.179000,14.702386,40.106802,77.594511
25143,298,82,129.807126,264.363306,8,20,35414,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.189807e+06,181363.305965,70.221738,14.734522,12312,0 days 00:00:00,119.604279,15.301029,39.603226,71.626426
25144,299,75,132.210510,264.802250,9,20,35474,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,12312,0 days 00:05:00,117.272271,15.240493,41.116443,71.624960


### Calculate distance relative to trollinde radar

In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[133.65217457503408,
 133.1777170980281,
 133.00050456755258,
 132.2997225053453,
 131.56306979814158,
 124.98202762426425,
 123.4845948958838,
 122.33876709003029,
 64.22017094091123,
 63.365712166846286]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 4.27 ms, sys: 0 ns, total: 4.27 ms
Wall time: 4.27 ms


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,311,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,3,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,463,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,3,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,610,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,3,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374


# close domain

In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_all = track[ mask_lat]
close_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
31,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,46.792859,13.245867,34.567482,28.852936
32,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,43.932525,12.974534,35.577388,32.832786
33,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,40.901944,12.780750,34.063904,29.847634
34,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,38.881259,12.708913,36.584485,20.893174
35,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,34.347343,12.845599,38.092506,18.904437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25104,297,74,275.687095,321.000000,2,15,35345,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.335687e+06,238000.000000,68.851046,15.818199,12273,0 days 00:05:00,44.049381,15.206985,38.598690,47.761373
25105,298,32,275.000000,321.000000,1,10,35390,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.335000e+06,238000.000000,68.857285,15.819899,12273,0 days 00:10:00,43.353280,15.301029,39.603226,29.851796
25119,297,27,239.000000,170.000000,1,10,35330,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.299000e+06,87000.000000,69.280754,12.167184,12286,0 days 00:00:00,151.242254,15.206985,38.598690,31.837359
25120,298,69,238.580361,173.129517,9,15,35408,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.298580e+06,90129.516623,69.283474,12.245472,12286,0 days 00:05:00,148.159082,15.301029,39.603226,43.776073


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_all['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_all[close_domain_all['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 944 ms, sys: 4.04 ms, total: 948 ms
Wall time: 971 ms


In [10]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_all['relative-time-to-radar'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_262789/3583987532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_all['relative-time-to-radar'] = cells_relative_times_to_radar_close.values


In [11]:
close_domain_all = close_domain_all.reset_index(drop=True)

In [12]:
close_domain_all.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),relative-time-to-radar
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,46.792859,13.245867,34.567482,28.852936,20
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,43.932525,12.974534,35.577388,32.832786,15
2,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,40.901944,12.780750,34.063904,29.847634,10
3,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,38.881259,12.708913,36.584485,20.893174,5
4,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,34.347343,12.845599,38.092506,18.904437,0


In [13]:
len(close_domain_all)

6920

# distant domain

In [14]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_all= track[ mask_lat ]
distant_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
19,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,197675.949726,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082
20,1,68,325.560542,282.342019,49,10,191,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.385561e+06,199342.018987,68.430539,14.776648,42,0 days 00:05:00,102.818375,12.974534,35.577388,58.710257
21,2,71,326.725123,282.815677,49,10,351,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.386725e+06,199815.677294,68.419587,14.785620,42,0 days 00:10:00,103.725137,12.780750,34.063904,58.711762
22,0,84,358.394575,40.394575,2,10,48,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.418395e+06,-42605.425332,68.203288,8.990711,48,0 days 00:00:00,305.706716,13.245867,34.567482,25.872526
23,1,94,361.014173,41.014173,2,10,203,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.421014e+06,-41985.827449,68.179476,9.006462,48,0 days 00:05:00,306.268077,12.974534,35.577388,14.926932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,299,43,144.743717,239.312025,4,15,35461,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,12303,0 days 00:05:00,122.013418,15.240493,41.116443,37.803245
25142,300,49,146.461260,240.148842,5,15,35519,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,12303,0 days 00:10:00,120.179000,14.702386,40.106802,77.594511
25143,298,82,129.807126,264.363306,8,20,35414,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.189807e+06,181363.305965,70.221738,14.734522,12312,0 days 00:00:00,119.604279,15.301029,39.603226,71.626426
25144,299,75,132.210510,264.802250,9,20,35474,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,12312,0 days 00:05:00,117.272271,15.240493,41.116443,71.624960


In [15]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_all['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_all[distant_domain_all['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 2.27 s, sys: 2 ms, total: 2.27 s
Wall time: 2.33 s


In [16]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_all['relative-time-to-radar'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_262789/643502393.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_all['relative-time-to-radar'] = cells_relative_times_to_radar_distant.values


In [17]:
distant_domain_all = distant_domain_all.reset_index(drop=True)


In [18]:
distant_domain_all.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),relative-time-to-radar
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,197675.949726,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,10
1,1,68,325.560542,282.342019,49,10,191,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.385561e+06,199342.018987,68.430539,14.776648,42,0 days 00:05:00,102.818375,12.974534,35.577388,58.710257,5
2,2,71,326.725123,282.815677,49,10,351,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.386725e+06,199815.677294,68.419587,14.785620,42,0 days 00:10:00,103.725137,12.780750,34.063904,58.711762,0
3,0,84,358.394575,40.394575,2,10,48,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.418395e+06,-42605.425332,68.203288,8.990711,48,0 days 00:00:00,305.706716,13.245867,34.567482,25.872526,15
4,1,94,361.014173,41.014173,2,10,203,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.421014e+06,-41985.827449,68.179476,9.006462,48,0 days 00:05:00,306.268077,12.974534,35.577388,14.926932,10


In [19]:
len(distant_domain_all)

18213

In [20]:
len(close_domain_all)

6920

In [21]:
len(track)

25146

In [22]:
len(close_domain_all) + len(distant_domain_all)

25133

In [23]:
distant_domain_all['latitude'].max(), distant_domain_all['latitude'].min(),close_domain_all['latitude'].max(), close_domain_all['latitude'].min()

(70.59773276537439, 67.88826554167575, 69.69399716573716, 68.78882750247271)

In [24]:
distant_domain_all['longitude'].max()

16.002843815564475

In [25]:
close_domain_all.to_csv('../saved-files/threshold-all/close-domain-threshold-all.csv', index=False)
distant_domain_all.to_csv('../saved-files/threshold-all/distant-domain-threshold-all.csv', index=False)

# END

In [42]:
# max_dBZ_10 = []
# for indexes, time in enumerate(np.unique(close_domain_10['relative-time-to-radar'])):
#     # Sorting based on same relative times
#     sort_relative_times_threshold_10 = close_domain_10[close_domain_10['relative-time-to-radar'] == time]
#     # Find each relative time associated with a particular reflectivity value
    
#     max_dbZ_threshold_10 = np.mean(sort_relative_times_threshold_10['max dBZ'])
#     max_dBZ_10.append(max_dbZ_threshold_10)